#### Batch Normalization

In [1]:
import keras.losses
from keras.layers import Flatten, Dense, BatchNormalization
from keras.models import Sequential

In [2]:
model = Sequential([
    Flatten(input_shape=[28, 28]),
    BatchNormalization(),
    Dense(300, activation="relu"),
    BatchNormalization(),
    Dense(100, activation="relu"),
    BatchNormalization(),
    Dense(10, activation="softmax")
])

2022-03-30 16:34:04.557341: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 16:34:04.636150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 16:34:04.636246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 16:34:04.636703: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [4]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

#### Gradient Clipping

In [5]:
from keras.optimizers import gradient_descent_v2

optimizer = gradient_descent_v2.SGD(clipvalue=1.0)

In [6]:
import numpy as np
from keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6)  # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2  # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32)  # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))


(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [7]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [8]:
X_train_A.shape

(43986, 28, 28)

In [9]:
X_train_B.shape

(200, 28, 28)

In [10]:
model_A = Sequential()
model_A.add(Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(Dense(n_hidden, activation="selu"))
model_A.add(Dense(8, activation="softmax"))

In [11]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [12]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
 157/1375 [==>...........................] - ETA: 1s - loss: 1.1933 - accuracy: 0.6083

2022-03-30 16:34:07.294717: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1375/1375 [==============================] - 2s 1ms/step - loss: 0.5712 - accuracy: 0.8127 - val_loss: 0.3849 - val_accuracy: 0.8662
Epoch 2/20
 403/1375 [=======>......................] - ETA: 0s - loss: 0.3836 - accuracy: 0.8672

KeyboardInterrupt: 

In [ ]:
model_A.save("my_model_A.h5")

In [ ]:
model_B = Sequential()
model_B.add(Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(Dense(n_hidden, activation="selu"))
model_B.add(Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_B.summary()

In [ ]:
from keras.models import load_model

model_A: Sequential = load_model("my_model_A.h5")
model_B_on_A = Sequential(model_A.layers[:-1])  # no output layer
model_B_on_A.add(Dense(1, activation="sigmoid"))

# Note that model_B_on_A and model_A share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build model_B_on_A on top of a clone of model_A:

In [ ]:
from keras.models import clone_model

model_A_clone = clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(Dense(1, activation="sigmoid"))

In [ ]:
for layer in model_B_on_A.layers[:-1]:  # Freezing all hidden layers
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

#### Momentum Optimization

In [ ]:
_ = gradient_descent_v2.SGD(momentum=0.9)

#### Nesterov Accelerated Gradient

In [ ]:
_ = gradient_descent_v2.SGD(momentum=0.9, nesterov=True)

#### AdaGrad

In [ ]:
from keras.optimizers import adagrad_v2

_ = adagrad_v2.Adagrad()  # not recommended for nn, since it can stop to early

#### RMSProp

In [ ]:
from keras.optimizers import rmsprop_v2

_ = rmsprop_v2.RMSprop(learning_rate=0.001, rho=0.9)

#### Adam and Nadam
nadam is Adam + Nesterov Trick

In [ ]:
from keras.optimizers import adam_v2

_ = adam_v2.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### Learning Rate Schedulers

#### Power Scheduling

In [ ]:
_ = gradient_descent_v2.SGD(learning_rate=0.01, decay=1e-4)  # decay adds power scheduling

#### Exponential scheduling

In [ ]:
# with hardcoded values
def exponential_decay_fn(epoch: int) -> float:
    return 0.01 * 0.1 ** (epoch / 20)


# with input values
def exponential_decay(lr0: float, s: int):
    def exponential_decay_fn(epoch: int) -> float:
        return lr0 * 0.1 ** (epoch / s)

    return exponential_decay_fn


exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

In [ ]:
model = Sequential([
    Flatten(input_shape=[28, 28]),
    Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 25

In [ ]:
from keras.callbacks import LearningRateScheduler

lr_scheduler = LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

In [ ]:
# decay will start at the beginning of epoch 0 instead of 1. Will rely upon existing lr (e.g. existing model)
def exponential_decay_fn(epoch: int, lr: float) -> float:
    return lr * 0.1 ** (epoch / 20)

#### Piecewise constant scheduling

In [ ]:
def piecewise_constant_fn(epoch: int) -> float:
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

#### Performance scheduling

In [ ]:
from keras.callbacks import ReduceLROnPlateau

_ = ReduceLROnPlateau(factor=0.5, patience=5)

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

s = 20 * len(X_train) // 32  # number of steps in 20 epochs (batch size = 32)
learning_rate = ExponentialDecay(0.01, s, 0.1)
optimizer = gradient_descent_v2.SGD(learning_rate)

#### Regularization

In [ ]:
#l1 and l2 regularization
from keras.regularizers import l1, l1_l2, l2

_ = Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2(0.01))
_ = Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l1(0.01))
_ = Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l1_l2(0.01))

#Creating preconfigured wrapper
from functools import partial

RegularizedDense = partial(Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = Sequential([
    Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
from keras.layers import Dropout

# Dropout
model = Sequential([
    Flatten(input_shape=[28, 28]),
    Dropout(rate=0.2),
    Dense(300, activation="elu", kernel_initializer="he_normal"),
    Dropout(rate=0.2),
    Dense(100, activation="elu", kernel_initializer="he_normal"),
    Dropout(rate=0.2),
    Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
# Monte Carlo (MC) Dropout
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)

In [ ]:
np.round(model.predict(X_test_scaled[:1]), 2)

In [ ]:
np.round(y_probas[:, :1], 2)

In [ ]:
np.round(y_proba[:1], 2)

In [ ]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

In [ ]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

In [ ]:
# Implementing mc without forcing training on all layers

class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [ ]:
# Max norm
from keras.constraints import max_norm

_ = Dense(100, activation="selu", kernel_initializer="lecun_normal", kernel_constraint=max_norm(1.))